In [77]:
import pandas as pd
import re
from collections import defaultdict

In [78]:
def load_survey_tables(file_path: str, sheet_name: str = "통계표"):
    """
    엑셀 파일에서 서울시 대기환경 조사 설문표를 파싱하여
    테이블, 질문 문장, 질문 키 목록을 반환합니다.

    Args:
        file_path (str): 엑셀 파일 경로
        sheet_name (str): 읽어올 시트 이름 (기본값: "통계표")

    Returns:
        tables (dict): {질문 key: 설문 결과 테이블}
        question_texts (dict): {질문 key: 질문 전체 문장}
        question_keys (list): 질문 key 목록
    """

    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

    pattern = r"^[A-Z]+\d*[-.]?\d*\."
    question_indices = df[df[0].astype(str).str.match(pattern)].index.tolist()

    tables = {}
    question_texts = {}
    question_keys = []
    key_counts = defaultdict(int)

    for i, start in enumerate(question_indices):
        end = question_indices[i + 1] if i + 1 < len(question_indices) else len(df)
        title = str(df.iloc[start, 0]).strip()

        match = re.match(pattern, title)
        if not match:
            continue
        base_key = match.group().rstrip(".")
        key_counts[base_key] += 1
        suffix = f"_{key_counts[base_key]}" if key_counts[base_key] > 1 else ""
        final_key = base_key + suffix

        question_texts[final_key] = title + "(전체 단위 : %)"
        question_keys.append(final_key)

        table = df.iloc[start + 1:end].reset_index(drop=True)
        if len(table) >= 2:
            new_columns = table.iloc[0].fillna('').astype(str) + " " + table.iloc[1].fillna('').astype(str)
            new_columns.iloc[0] = "대분류"
            if len(new_columns) > 1:
                new_columns.iloc[1] = "소분류"
            table = table.drop([0, 1]).reset_index(drop=True)
            table.columns = new_columns
            table["대분류"] = table["대분류"].ffill()

            table = table.dropna(axis=1, how='all')
            table = table.dropna(axis=0, how='all')

            table = table.drop(index=0).reset_index(drop=True)
            if len(table) > 2:
                table = table.iloc[:-2].reset_index(drop=True)

            # ✅ 소수점 첫째 자리까지 반올림 (숫자형으로 변환 후 처리)
            for col in table.columns:
                try:
                    numeric_col = pd.to_numeric(table[col], errors='coerce')
                    if numeric_col.notna().any():
                        table[col] = numeric_col.round(1)
                except:
                    continue

            tables[final_key] = table

    return tables, question_texts, question_keys

In [76]:
# 함수 호출
tables, question_texts, question_keys = load_survey_tables("통계표_수정_서울시 대기환경 시민인식 조사_250421(작업용).xlsx")

# 확인
print("질문 Key 목록:", question_keys)
print("질문:", question_texts.get("A2"))
tables["A4"]

질문 Key 목록: ['A1', 'A2', 'A3', 'A3_2', 'A3_3', 'A4', 'A4-1', 'A4-2', 'B1', 'B1-1', 'B1-2', 'B2', 'B3', 'B3_2', 'B3_3', 'B4', 'B4_2', 'B5', 'B5_2', 'B6', 'B6_2', 'B7', 'B8', 'B8_2', 'B9', 'B9_2', 'B10', 'B10_2', 'B11', 'B12', 'B13', 'B14', 'B14_2', 'B14_3', 'B14_4']
질문: A2. 현재 서울시의 전반적인 대기오염 수준은 어느 정도라고 생각하십니까?(전체 단위 : %)


,대분류,소분류,사례수,서울시 대기오염 수준 개선 정도 전혀 개선되지 않았다,대체로 개선되지 않았다,과거와 비슷하다,대체로 개선되었다,매우 개선되었다,개선되지 않았다 %,과거와 비슷하다 %,개선되었다 %,평균(4점척도)
0,전 체,NaN,1000,21.6,32.6,33.9,11.5,0.4,54.2,33.9,11.9,2.4
1,성별,남성,494,21.1,27.9,35.6,14.8,0.6,49.0,35.6,15.4,2.5
2,성별,여성,506,22.1,37.2,32.2,8.3,0.2,59.3,32.2,8.5,2.3
3,연령,20대,181,21.0,38.1,33.1,6.6,1.1,59.1,33.1,7.7,2.3
4,연령,30대,244,27.9,29.9,32.8,9.0,0.4,57.8,32.8,9.4,2.2
5,연령,40대,205,24.9,32.2,35.1,7.8,0.0,57.1,35.1,7.8,2.3
6,연령,50대,167,20.4,31.1,36.5,12.0,0.0,51.5,36.5,12.0,2.4
7,연령,60대이상,203,12.3,32.5,32.5,22.2,0.5,44.8,32.5,22.7,2.7
8,생활권역,도심권,51,21.6,27.5,31.4,19.6,0.0,49.0,31.4,19.6,2.5
9,생활권역,동북권,293,26.6,28.0,34.8,10.2,0.3,54.6,34.8,10.6,2.3
